This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Find a build order given a list of projects and dependencies.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Is it possible to have a cyclic graph as the input?
    * Yes
* Can we assume we already have Graph and Node classes?
    * Yes
* Can we assume this is a connected graph?
    * Yes
* Can we assume the inputs are valid?
    * Yes
* Can we assume this fits memory?
    * Yes

## Test Cases

* projects: a, b, c, d, e, f, g
* dependencies: (d, g), (f, c), (f, b), (f, a), (c, a), (b, a), (a, e), (b, e)
* output: d, f, c, b, g, a, e

Note: Edge direction is down
<pre>
    f     d
   /|\    |
  c | b   g
   \|/|
    a |
    |/
    e
</pre>

Test a graph with a cycle, output should be None

## Algorithm

Refer to the [Solution Notebook](http://nbviewer.jupyter.org/github/donnemartin/interactive-coding-challenges/blob/master/graphs_trees/graph_build_order/build_order_solution.ipynb).  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

**Solution**: This is basically a Topsort (topological sort).
topsort:
1. sort the verticies ascendingly by # inbound edges.
2. all verticies with 0 inbound edges can be used now.
3. remove those verticies from the graph.
4. repeat until the graph is empty


Note: topsort doesn't allow cycles in graphs.

## Code

In [1]:
class Dependency(object):

    def __init__(self, node_key_before, node_key_after):
        self.node_key_before = node_key_before
        self.node_key_after = node_key_after

In [2]:
# %load ../graph/graph.py
from enum import Enum  # Python 2 users: Run pip install enum34


class State(Enum):
    unvisited = 0
    visiting = 1
    visited = 2


class Node:

    def __init__(self, key):
        self.key = key
        self.visit_state = State.unvisited
        self.incoming_edges = 0
        self.adj_nodes = {}  # Key = key, val = Node
        self.adj_weights = {}  # Key = key, val = weight

    def __repr__(self):
        return str(self.key)

    def __lt__(self, other):
        return self.key < other.key

    def add_neighbor(self, neighbor, weight=0):
        if neighbor is None or weight is None:
            raise TypeError('neighbor or weight cannot be None')
        neighbor.incoming_edges += 1
        self.adj_weights[neighbor.key] = weight
        self.adj_nodes[neighbor.key] = neighbor

    def remove_neighbor(self, neighbor):
        if neighbor is None:
            raise TypeError('neighbor cannot be None')
        if neighbor.key not in self.adj_nodes:
            raise KeyError('neighbor not found')
        neighbor.incoming_edges -= 1
        del self.adj_weights[neighbor.key]
        del self.adj_nodes[neighbor.key]


class Graph:

    def __init__(self):
        self.nodes = {}  # Key = key, val = Node

    def add_node(self, key):
        if key is None:
            raise TypeError('key cannot be None')
        if key not in self.nodes:
            self.nodes[key] = Node(key)
        return self.nodes[key]

    def add_edge(self, source_key, dest_key, weight=0):
        if source_key is None or dest_key is None:
            raise KeyError('Invalid key')
        if source_key not in self.nodes:
            self.add_node(source_key)
        if dest_key not in self.nodes:
            self.add_node(dest_key)
        self.nodes[source_key].add_neighbor(self.nodes[dest_key], weight)

    def add_undirected_edge(self, src_key, dst_key, weight=0):
        if src_key is None or dst_key is None:
            raise TypeError('key cannot be None')
        self.add_edge(src_key, dst_key, weight)
        self.add_edge(dst_key, src_key, weight)

In [3]:
class BuildOrder(object):

    def __init__(self, dependencies):
        # each dependency object is a directed edge.
        # each edge contains a 'before' and 'after', which are nodes.
        # Build a graph
        self.graph = Graph()
        for edge in dependencies:
            self.graph.add_node(edge.node_key_before)
            self.graph.add_node(edge.node_key_after)
            self.graph.add_edge(edge.node_key_before, edge.node_key_after)
        self.n_nodes = len(self.graph.nodes)
        
    def find_build_order(self):
        result = []
        iterations = 0
        
        while len(self.graph.nodes) > 0:
            if iterations > self.n_nodes:
                return None
            # sort verticies by n_incoming edges
            nodes_by_incoming_edges = sorted(self.graph.nodes.values(), key=lambda x: x.incoming_edges)
            n_incoming = [i.incoming_edges for i in nodes_by_incoming_edges]
            n_incoming_and_nodes = zip(n_incoming, nodes_by_incoming_edges)

            # use all verticies with 0 incoming edges now
            # remove them from the graph
            for count, node in n_incoming_and_nodes:
                if count > 0:
                    break
                result.append(node)
                for nbr in node.adj_nodes.values():
                    node.remove_neighbor(nbr)
                self.graph.nodes.pop(node.key)
            iterations += 1
        return result
        

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [4]:
# %load test_build_order.py
from nose.tools import assert_equal
from nose.tools import assert_true


class TestBuildOrder(object):

    def __init__(self):
        self.dependencies = [
            Dependency('d', 'g'),
            Dependency('f', 'c'),
            Dependency('f', 'b'),
            Dependency('f', 'a'),
            Dependency('c', 'a'),
            Dependency('b', 'a'),
            Dependency('a', 'e'),
            Dependency('b', 'e'),
        ]

    def test_build_order(self):
        build_order = BuildOrder(self.dependencies)
        processed_nodes = build_order.find_build_order()

        expected_result0 = ('d', 'f')
        expected_result1 = ('c', 'b', 'g')
        assert_true(processed_nodes[0].key in expected_result0)
        assert_true(processed_nodes[1].key in expected_result0)
        assert_true(processed_nodes[2].key in expected_result1)
        assert_true(processed_nodes[3].key in expected_result1)
        assert_true(processed_nodes[4].key in expected_result1)
        assert_true(processed_nodes[5].key is 'a')
        assert_true(processed_nodes[6].key is 'e')

        print('Success: test_build_order')

    def test_build_order_circular(self):
        self.dependencies.append(Dependency('e', 'f'))
        build_order = BuildOrder(self.dependencies)
        processed_nodes = build_order.find_build_order()
        assert_true(processed_nodes is None)

        print('Success: test_build_order_circular')


def main():
    test = TestBuildOrder()
    test.test_build_order()
    test.test_build_order_circular()


if __name__ == '__main__':
    main()

Success: test_build_order
Success: test_build_order_circular


## Solution Notebook

Review the [Solution Notebook](https://github.com/donnemartin/interactive-coding-challenges/graphs_trees/build_order/build_order_solution.ipynb) for a discussion on algorithms and code solutions.